![dvd_image](dvd_image.jpg)

A DVD rental company needs your help! They want to figure out how many days a customer will rent a DVD for based on some features and has approached you for help. They want you to try out some regression models which will help predict the number of days a customer will rent a DVD for. The company wants a model which yeilds a MSE of 3 or less on a test set. The model you make will help the company become more efficient inventory planning.

The data they provided is in the csv file `rental_info.csv`. It has the following features:
- `"rental_date"`: The date (and time) the customer rents the DVD.
- `"return_date"`: The date (and time) the customer returns the DVD.
- `"amount"`: The amount paid by the customer for renting the DVD.
- `"amount_2"`: The square of `"amount"`.
- `"rental_rate"`: The rate at which the DVD is rented for.
- `"rental_rate_2"`: The square of `"rental_rate"`.
- `"release_year"`: The year the movie being rented was released.
- `"length"`: Lenght of the movie being rented, in minuites.
- `"length_2"`: The square of `"length"`.
- `"replacement_cost"`: The amount it will cost the company to replace the DVD.
- `"special_features"`: Any special features, for example trailers/deleted scenes that the DVD also has.
- `"NC-17"`, `"PG"`, `"PG-13"`, `"R"`: These columns are dummy variables of the rating of the movie. It takes the value 1 if the move is rated as the column name and 0 otherwise. For your convinience, the reference dummy has already been dropped.

In [3]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import matplotlib.pyplot as plt

# Import any additional modules and start coding below

In [4]:
# Read the csv file
rental_df = pd.read_csv("rental_info.csv")

In [258]:
rental_df.head()

,rental_date,return_date,amount,release_year,rental_rate,length,replacement_cost,special_features,NC-17,PG,PG-13,R,amount_2,length_2,rental_rate_2
0,2005-05-25 02:54:33+00:00,2005-05-28 23:40:33+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
1,2005-06-15 23:19:16+00:00,2005-06-18 19:24:16+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
2,2005-07-10 04:27:45+00:00,2005-07-17 10:11:45+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
3,2005-07-31 12:06:41+00:00,2005-08-02 14:30:41+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
4,2005-08-19 12:30:04+00:00,2005-08-23 13:35:04+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401


In [259]:
rental_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15861 entries, 0 to 15860
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   rental_date       15861 non-null  object 
 1   return_date       15861 non-null  object 
 2   amount            15861 non-null  float64
 3   release_year      15861 non-null  float64
 4   rental_rate       15861 non-null  float64
 5   length            15861 non-null  float64
 6   replacement_cost  15861 non-null  float64
 7   special_features  15861 non-null  object 
 8   NC-17             15861 non-null  int64  
 9   PG                15861 non-null  int64  
 10  PG-13             15861 non-null  int64  
 11  R                 15861 non-null  int64  
 12  amount_2          15861 non-null  float64
 13  length_2          15861 non-null  float64
 14  rental_rate_2     15861 non-null  float64
dtypes: float64(8), int64(4), object(3)
memory usage: 1.8+ MB


In [5]:
# Create "rental_length_days" column
rental_df["rental_date"] = pd.to_datetime(rental_df["rental_date"])
rental_df["return_date"] = pd.to_datetime(rental_df["return_date"])

rental_df["rental_length_days"] = (rental_df["return_date"] - rental_df["rental_date"]).dt.days

In [261]:
rental_df.head()

,rental_date,return_date,amount,release_year,rental_rate,length,replacement_cost,special_features,NC-17,PG,PG-13,R,amount_2,length_2,rental_rate_2,rental_length_days
0,2005-05-25 02:54:33+00:00,2005-05-28 23:40:33+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401,3
1,2005-06-15 23:19:16+00:00,2005-06-18 19:24:16+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401,2
2,2005-07-10 04:27:45+00:00,2005-07-17 10:11:45+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401,7
3,2005-07-31 12:06:41+00:00,2005-08-02 14:30:41+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401,2
4,2005-08-19 12:30:04+00:00,2005-08-23 13:35:04+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401,4


## Creating Dummies

In [6]:
rental_df["deleted_scenes"] = np.where(rental_df["special_features"].str.contains("Deleted Scenes"), 1, 0)
rental_df["behind_the_scenes"] = np.where(rental_df["special_features"].str.contains("Behind the Scenes"), 1, 0)
rental_df.head()

,rental_date,return_date,amount,release_year,rental_rate,length,replacement_cost,special_features,NC-17,PG,PG-13,R,amount_2,length_2,rental_rate_2,rental_length_days,deleted_scenes,behind_the_scenes
0,2005-05-25 02:54:33+00:00,2005-05-28 23:40:33+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401,3,0,1
1,2005-06-15 23:19:16+00:00,2005-06-18 19:24:16+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401,2,0,1
2,2005-07-10 04:27:45+00:00,2005-07-17 10:11:45+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401,7,0,1
3,2005-07-31 12:06:41+00:00,2005-08-02 14:30:41+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401,2,0,1
4,2005-08-19 12:30:04+00:00,2005-08-23 13:35:04+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401,4,0,1


In [7]:
rental_df["behind_the_scenes"].value_counts()

1    8507
0    7354
Name: behind_the_scenes, dtype: int64

In [264]:
# rental_df.corr()

In [265]:
# sns.pairplot(data=rental_df)
# plt.show

In [266]:
# rental_df.var()

## Feature Selection

In [267]:
X = rental_df.drop(columns=["rental_length_days", "return_date", "rental_date", "special_features"])
X.head()

,amount,release_year,rental_rate,length,replacement_cost,NC-17,PG,PG-13,R,amount_2,length_2,rental_rate_2,deleted_scenes,behind_the_scenes
0,2.99,2005.0,2.99,126.0,16.99,0,0,0,1,8.9401,15876.0,8.9401,0,1
1,2.99,2005.0,2.99,126.0,16.99,0,0,0,1,8.9401,15876.0,8.9401,0,1
2,2.99,2005.0,2.99,126.0,16.99,0,0,0,1,8.9401,15876.0,8.9401,0,1
3,2.99,2005.0,2.99,126.0,16.99,0,0,0,1,8.9401,15876.0,8.9401,0,1
4,2.99,2005.0,2.99,126.0,16.99,0,0,0,1,8.9401,15876.0,8.9401,0,1


In [268]:
X.shape

(15861, 14)

In [269]:
y = rental_df["rental_length_days"]
y.head()

0    3
1    2
2    7
3    2
4    4
Name: rental_length_days, dtype: int64

## Splitting the data

In [270]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=9)

## Standardizing the data

In [271]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

## Model Selection

In [272]:
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import mean_squared_error

lin_reg = LinearRegression()
lasso = Lasso(random_state=9)

lin_reg.fit(X_train_scaled, y_train)
y_pred_lin = lin_reg.predict(X_test_scaled)
print(f"Lin_reg MSE= {mean_squared_error(y_test, y_pred_lin)}")

lasso.fit(X_train_scaled, y_train)
y_pred_lasso = lasso.predict(X_test_scaled)
print(f"Lasso MSE= {mean_squared_error(y_test, y_pred_lasso)}")

Lin_reg MSE= 2.9417238646975976
Lasso MSE= 5.9495850986576295


In [273]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
param_grid={'alpha': np.linspace(0.0001, 1, 50)}

gridsearch = GridSearchCV(lasso, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
gridsearch.fit(X_train_scaled, y_train)

GridSearchCV(cv=5, estimator=Lasso(random_state=9), n_jobs=-1,
             param_grid={'alpha': array([1.00000000e-04, 2.05061224e-02, 4.09122449e-02, 6.13183673e-02,
       8.17244898e-02, 1.02130612e-01, 1.22536735e-01, 1.42942857e-01,
       1.63348980e-01, 1.83755102e-01, 2.04161224e-01, 2.24567347e-01,
       2.44973469e-01, 2.65379592e-01, 2.85785714e-01, 3.06191837e-01,
       3.26597959e-01, 3.470040...
       5.71471429e-01, 5.91877551e-01, 6.12283673e-01, 6.32689796e-01,
       6.53095918e-01, 6.73502041e-01, 6.93908163e-01, 7.14314286e-01,
       7.34720408e-01, 7.55126531e-01, 7.75532653e-01, 7.95938776e-01,
       8.16344898e-01, 8.36751020e-01, 8.57157143e-01, 8.77563265e-01,
       8.97969388e-01, 9.18375510e-01, 9.38781633e-01, 9.59187755e-01,
       9.79593878e-01, 1.00000000e+00])},
             scoring='neg_mean_squared_error')

In [274]:
best_mse = -gridsearch.best_score_
best_model = gridsearch.best_estimator_
print(best_mse)

2.8490659482968193


In [275]:
param_grid = {"alpha": np.arange(0.0001, 1, 10),"solver": ["sag", "lsqr"]}
ridge = Ridge(random_state=9)
ridge_cv = GridSearchCV(ridge, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
ridge_cv.fit(X_train_scaled, y_train)


GridSearchCV(cv=5, estimator=Ridge(random_state=9), n_jobs=-1,
             param_grid={'alpha': array([0.0001]), 'solver': ['sag', 'lsqr']},
             scoring='neg_mean_squared_error')

In [276]:
best_mse = -ridge_cv.best_score_
best_model = ridge_cv.best_estimator_
print(best_mse)

2.8490604588098245


In [277]:
from sklearn.tree import DecisionTreeRegressor
regr = DecisionTreeRegressor(random_state=9)

regr.fit(X_train_scaled, y_train)
y_pred_regr = regr.predict(X_test_scaled)
print(f"Decision Tree MSE= {mean_squared_error(y_test, y_pred_regr)}")

Decision Tree MSE= 2.1673584985496035


In [278]:
from sklearn.ensemble import RandomForestRegressor
forest = RandomForestRegressor(random_state=9)

forest.fit(X_train_scaled, y_train)
y_pred_forest = forest.predict(X_test_scaled)
print(f"Forest MSE= {mean_squared_error(y_test, y_pred_forest)}")

Forest MSE= 2.0305235428385586


In [279]:
param_distributions = {
    'n_estimators': np.linspace(100, 200, 10, dtype=int),  # Should be a list/array of ints
    'max_features': ['auto', 'sqrt'],
    'max_depth': np.linspace(5, 20, 10, dtype=int),        # Should be a list/array of ints or None
}

rf_random = RandomizedSearchCV(
    estimator=forest, 
    param_distributions=param_distributions, 
    n_iter=10, 
    cv=3, 
    scoring='neg_mean_squared_error',
    verbose=2, 
    random_state=42, 
    n_jobs=-1
)

rf_random.fit(X_train_scaled, y_train)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


RandomizedSearchCV(cv=3, estimator=RandomForestRegressor(random_state=9),
                   n_jobs=-1,
                   param_distributions={'max_depth': array([ 5,  6,  8, 10, 11, 13, 15, 16, 18, 20]),
                                        'max_features': ['auto', 'sqrt'],
                                        'n_estimators': array([100, 111, 122, 133, 144, 155, 166, 177, 188, 200])},
                   random_state=42, scoring='neg_mean_squared_error',
                   verbose=2)

In [280]:
y_pred_rf = rf_random.predict(X_test_scaled)
rf_mse = mean_squared_error(y_test, y_pred_rf)
print(f"rf MSE= {rf_mse}")

rf MSE= 2.024043908439146


In [281]:
best_mse = rf_mse
best_model = rf_random.best_estimator_
print(best_model)

RandomForestRegressor(max_depth=13, max_features='sqrt', n_estimators=155,
                      random_state=9)
